In [ ]:
# --- core ---
import os
import asyncio
import datetime as dt
from zoneinfo import ZoneInfo
import nest_asyncio
import math
nest_asyncio.apply()   # only needed if running in Jupyter

# --- alpaca trading / data ---
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, StopOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, StockLatestTradeRequest
from alpaca.data.live import StockDataStream
from alpaca.data.enums import DataFeed

# --- optional (only if you want data analysis or plotting) ---
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import yfinance as yf
# import backtrader as bt
# import pytz
#api_key="PKB9SXQSORHZSB1VIXY0"
#secret_key="q4n01bamgGErL7p261A7mu5UvHZOOTO2i6rSsXgJ"



In [ ]:
# latest code should work fine - test on 08/19 

API_KEY    = "PKB9SXQSORHZSB1VIXY0"
API_SECRET = "q4n01bamgGErL7p261A7mu5UvHZOOTO2i6rSsXgJ"

trading = TradingClient(API_KEY, API_SECRET, paper=True)
client = StockHistoricalDataClient(API_KEY, API_SECRET)

symbol = "SNGX"

# look back a week to be safe
start = dt.datetime.utcnow() - dt.timedelta(days=7)

req = StockBarsRequest(
    symbol_or_symbols=symbol,
    timeframe=TimeFrame.Day,
    start=start,
)

bars = client.get_stock_bars(req).df

# If multiple days, take the second-to-last bar = yesterday's close
bars = bars.xs(symbol, level="symbol") if "symbol" in bars.index.names else bars
bars = bars.sort_index()

if len(bars) >= 2:
    prior_close = float(bars["close"].iloc[-2])
    prior_ts    = bars.index[-2] 
else:
    prior_close = float(bars["close"].iloc[-1])
    prior_ts    = bars.index[-1] 

print(f"Yesterday's close for {symbol}: {prior_close} @ {prior_ts}")
#print(bars)

state = {
    "activated": False,
    "shares": 0,         # negative = short
    "proceeds": 0.0,     # sum(|qty| * price) for shorts
    "mwap": 0.0,         # proceeds / |shares|
}

activated = False
stream = StockDataStream(API_KEY, API_SECRET, feed=DataFeed.IEX)  # or SIP if you have subscription


async def on_trade(trade):
    print("TICK!")
    if trade.symbol != symbol:
        return

    price = float(trade.price)
    ts    = trade.timestamp
    print(f"Live tick: {trade.symbol} {price} at {ts}")

    # 1) Trigger
    if (not state["activated"]) and price / prior_close >= 1.5:
        state["activated"] = True

        init_qty = 100                     # initial short (shares)
        # place initial short (SELL init_qty)
        order = trading.submit_order(MarketOrderRequest(
            symbol=symbol,
            qty=init_qty,
            side=OrderSide.SELL,
            time_in_force=TimeInForce.DAY
        ))
        # update short state: shares negative
        state["shares"]   -= init_qty
        state["proceeds"] += init_qty * price
        q = abs(state["shares"])
        state["mwap"]     = state["proceeds"] / q
        print(f"[ENTER] short {init_qty} @ ~{price:.4f} | pos={state['shares']} | MWAP={state['mwap']:.4f}")

        return  # done on this tick

    # 2) Ladder adds (only after activated and already short)
    if state["activated"] and state["shares"] < 0:
        target = 0.90 * price
        M = state["mwap"]
        if M < target:
            # dq = q*(T - M) / (P - T)  ; add_shares = int(dq) >= 1
            q = abs(state["shares"])
            if price > target:  # guard divide by zero
                dq = (q * (target - M)) / (price - target)
                add = max(1, int(math.floor(dq)))  # integer shares to ADD
                # submit ONLY the delta
                order = trading.submit_order(MarketOrderRequest(
                    symbol=symbol,
                    qty=add,
                    side=OrderSide.SELL,
                    time_in_force=TimeInForce.DAY
                ))
                # update state
                state["shares"]   -= add
                state["proceeds"] += add * price
                state["mwap"]      = state["proceeds"] / abs(state["shares"])
                print(f"[LADDER] add {add} @ ~{price:.4f} | pos={state['shares']} | MWAP={state['mwap']:.4f}")


stream.subscribe_trades(on_trade, symbol)

# run
import asyncio
asyncio.run(stream._run_forever())

In [ ]:
await stream.stop()
await stream_task
print("Stream stopped.")